In [1]:
# SPDX-License-Identifier: GPL-3.0-or-later

In [1]:
import numpy
import pandas
import os
import h5py

import conntility
import bluepysnap as bluepy

from matplotlib import pyplot as plt

fn_sim = "../sscxSimulation/simulation_config.json"
population = "S1nonbarrel_neurons"
pop_filter = [  # Here: Only plot the excitatory population
    {
        "column": "synapse_class",
        "values": ["INH"]
    }
]
bin_sz = 10 # Size of time bins

sim = bluepy.Simulation(fn_sim)
spk_fn = os.path.join(sim.config["output"]["output_dir"], sim.config["output"]["spikes_file"])
circ = sim.circuit
spk_h5 = h5py.File(spk_fn, "r")




## Load spikes

and format them into a useful representation.
Target format is a pandas Series, indexed by neuron id, values corresponding to the time bin they fall into.

Specifically, time bins are represented by the time in ms they start. This allows us to remove duplicate spikes in a bin, i.e. when a neuron spikes more than one time in a bin, only the first one is kept. Reasoning: We want to plot what fraction of the population is recruited, not how strongly a given neuron is recruited.

In [2]:
bins = numpy.arange(0, 10000 + bin_sz, bin_sz)

t = spk_h5["spikes"][population]["timestamps"][:]
tbins = bin_sz * (numpy.digitize(t, bins=bins) - 1)
spks = pandas.Series(tbins, name="tbins",
                     index=pandas.Index(spk_h5["spikes"][population]["node_ids"][:], name="node_ids"))

spks = spks.reset_index().drop_duplicates().set_index("node_ids", drop=True)
spks.head()

,tbins
node_ids,
3600059,20
3508860,20
3594234,20
3596862,20
3597829,20


# Load neuron information
We load the neuron, apply the specified filter, and group them into hexagonal bins with 50 um diameter

In [3]:
load_cfg = {
    "loading":{    
        "properties": ["x", "y", "z", "ss_flat_x", "ss_flat_y", "synapse_class"],
        "base_target": "Mosaic",
    },
    "filtering": pop_filter,
    "grouping": [
        {
            "method": "group_by_grid",
            "columns": ["ss_flat_x", "ss_flat_y"],
            "args": [50.0],
            "kwargs":{
                "prefix": "hex-"                
            }
        }
    ]
}

props = ["x", "y", "z", "ss_flat_x", "ss_flat_y", "synapse_class"]

nrn = conntility.circuit_models.neuron_groups.load_group_filter(
    circ,
    load_cfg,
    node_population=population
)

Rotation errors: min: 0.0, median: 0.09388555231229005, mean: 0.13639662907091096, std: 0.1572698939879801, max: 2.0


/gpfs/bbp.cscs.ch/home/tharayil/conntilitEnv/lib/python3.11/site-packages/conntility/circuit_models/neuron_groups/tessellate.py:381: RuntimeWarning: invalid value encountered in cast
  scaled_u = np.array(np.floor(uvs.u.values / self._side), dtype=int)
/gpfs/bbp.cscs.ch/home/tharayil/conntilitEnv/lib/python3.11/site-packages/conntility/circuit_models/neuron_groups/tessellate.py:382: RuntimeWarning: invalid value encountered in cast
  scaled_v = np.array(np.floor(uvs.v.values / self._side), dtype=int)


# Preliminary calculations
We create a number of lookups:
- Lookup from the name of a hexagon group to the location of the hexagon in flat space
- Lookup from neuron id to name of the hexagon group it belongs to

Then we filter the loaded spikes to only keep the ones we have a lookup for. That applies the specified filter also to the spikes. After this, we represent the spikes no longer by the neuron id of the spiking neuron, but only by the hexagon group it belongs to!

Finally, we count the number of neurons in each hexagon group, to be used for normalization.

In [4]:
# Position lookup
positions = nrn.set_index("hex-subtarget")[["hex-x", "hex-y"]].drop_duplicates()
positions = positions.loc[positions["hex-x"] > -1E10]

# hexagon group lookup
lo = nrn.set_index("node_ids")["hex-subtarget"]

# Discard spikes of neurons that have not been loaded, i.e. filtered out
valid = numpy.in1d(spks.index, lo.index)
spks = spks[valid]
spks.index = pandas.Index(lo[spks.index].values, name="group")

# Number of neurons in each hexagon subtarget
grp_counts = nrn.groupby("hex-subtarget")["x"].count()

# Actual calculation
Now we count the number of neurons recruited in each combination of hexagon and time bin. 
The result is normalized by the total number of neurons in the corresponding hexagon. That is, the result is the percentage of neurons in a hexagon group that is recruited in the time bin, i.e. fires at least once.

In [5]:
def bin_and_counts(grp):
    x = grp.index[0]
    H = numpy.histogram(grp.values, bins=bins)[0]
    return pandas.Series(H / grp_counts[x], index=pandas.Index(bins[:-1], name="tbins"))

H = spks.groupby("group").apply(bin_and_counts) * 100

# Restrict to hexagon groups we have locations for.
H = H.loc[H.index.intersection(positions.index)]
# Ensure positions and recruitment vectors are in the same order. That way we can easily plot them.
positions = positions.loc[H.index]

# Plotting

In [6]:
import tqdm
figsz = (5, 4)
plt_bins = numpy.arange(3000, 3760, 10)
out_dir = "./movie_inh"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

    
for i, t_tgt in tqdm.tqdm(enumerate(plt_bins)):
    fig = plt.figure(figsize=figsz)
    ax = fig.gca()
    s = ax.scatter(positions["hex-x"].values, positions["hex-y"].values, marker="h", s=8,
               c=H[t_tgt].values, clim=[0, 50])
    plt.colorbar(s, shrink=0.7, label="Perc. recruited")
    plt.axis("equal")
    ax.set_frame_on(False)
    ax.set_xticks([]); ax.set_yticks([])
    
    fig.savefig(os.path.join(out_dir, "frame-{i:03}.png".format(i=i)))
    plt.close(fig)

76it [00:10,  7.01it/s]
